# Otimizando os hiperparametros do random forest

Nesse notebook iremos otimizar o hiperparametros do random forest usando com técnicas com o gridsearch e o random search e faremos a compararção se melhorou a perfomace.

In [1]:
# Importando os módulos
import numpy as np 
import pandas as pd 
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# utilizarei esse parametro só para comparar com naive bayes
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV , RandomizedSearchCV 



In [2]:
# Importando o dataset
heart_failure = pd.read_csv('heart_failure_clinical_records_dataset.csv')

In [3]:
# fazendo as separação dos atributos e do target
x = heart_failure.drop(['DEATH_EVENT'],axis=1).values
y = heart_failure.DEATH_EVENT.values

# Normalizando os dados
x_std = StandardScaler().fit_transform(x)

In [4]:
# usando o metos undersampling para tornar os dados menos enviesados
nm = RandomUnderSampler(random_state = 10)

x_nm, y_nm = nm.fit_resample(x_std, y)

## Aplicando o gridsearch para otimizar os hiperparametros

In [5]:
parametros = {'n_estimators':np.arange(20,31,1),'criterion':['gini', 'entropy', 'log_loss'],'max_features':['sqrt','log2',None],
'bootstrap':[True,False],'warm_start':[True,False],'class_weight':['balanced','balanced_subsample']}

In [6]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(),param_grid=parametros)
grid_search.fit(x_nm,y_nm)
melhores_parametros = grid_search.best_params_
melhor_resultado = grid_search.best_score_

print(melhores_parametros)
print(melhor_resultado)

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the r

{'bootstrap': False, 'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 20, 'warm_start': True}
0.7960863697705802


Foi feito primeiro o n_estimators de 1 até a 10 e o melhor resultado foi esse:


O melhor parametro que foi mostrado foram
{'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 7, 'warm_start': True}
com score de 0.796221322537112

e depois feit o n_estimators de 20 até a 30 e o melhor resultado foi esse:

{'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 21, 'warm_start': True}
com score de 0.7856950067476384

logo usaremos o primeiro

In [15]:

nome_metricas = ['accuracy', 'precision_macro', 'recall_macro']

ran_f = RandomForestClassifier(n_estimators=7,criterion='entropy',max_features='sqrt',warm_start=True,bootstrap=False,
class_weight='balanced')

metricas_ran = cross_validate(ran_f,x_nm, y_nm, cv=7, scoring=nome_metricas)
for met in metricas_ran:
  print(f'-{met}')
  print(f"-- {metricas_ran[met]}")
  media = np.mean(metricas_ran[met])
  desvio = np.std(metricas_ran[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the r

-fit_time
-- [0.1126821  0.05256772 0.0510838  0.07056689 0.0776     0.05734444
 0.03505993]
Média do fit_time: 0.06527212687901088
Desvio 0.023194989145349083
Intervalo [0.019,0.112]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.01474929 0.0184381  0.01438856 0.03292799 0.0202837  0.02300096
 0.02576733]
Média do score_time: 0.021365131650652205
Desvio 0.006070230059472847
Intervalo [0.009,0.034]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.92857143 0.75       0.92857143 0.92592593 0.81481481 0.7037037
 0.48148148]
Média do test_accuracy: 0.7904383975812549
Desvio 0.15199863518278794
Intervalo [0.486,1.094]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision_macro
-- [0.9375     0.83333333 0.92857143 0.92582418 0.81666667 0.75328947
 0.48529412]
Média do test_precision_macro: 0.811497027960982
Desvio 0.14781932429649772
Intervalo [0.516,1.107]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the r

Comparando as métricas, temos que
- random forest *sem otimização*:
    - acuracia: 0,758
    - precisão: 0,776
    - recall: 0,761

- random forest *com otimização de grid search*:
    - acuracia: 0.790
    - precisão: 0.811
    - recall: 0.791

Como podemos ver não tivemos uma melhora em cada métrica

## Aplicando o randomsearch para otimizar os hiperparametros

In [8]:
parametros2 = {'n_estimators':np.arange(20,31,1),'criterion':['gini', 'entropy', 'log_loss'],'max_features':['sqrt','log2',None],
'bootstrap':[True,False],'warm_start':[True,False],'class_weight':['balanced','balanced_subsample']}

In [9]:
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),param_distributions=parametros2)
random_search.fit(x_nm,y_nm)
melhores_parametros2 = random_search.best_params_
melhor_resultado2 = random_search.best_score_

print(melhores_parametros2)
print(melhor_resultado2)

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the r

{'warm_start': False, 'n_estimators': 26, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced_subsample', 'bootstrap': True}
0.7906882591093117


de n estimator de 1 a 10 , tivemos os seguinets resultados:
{'warm_start': True, 'n_estimators': 6, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': True}
com score de 0.769770580296896

de n estimator de 10 a 20
{'warm_start': False, 'n_estimators': 17, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': True}
com score de 0.7751686909581645

de 20 a 30
{'warm_start': False, 'n_estimators': 29, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced_subsample', 'bootstrap': False}
 co score de 0.7701754385964912

de n estimator de 30 a 60
{'warm_start': True, 'n_estimators': 40, 'max_features': 'log2', 'criterion': 'entropy', 'class_weight': 'balanced_subsample', 'bootstrap': False}
com score de 0.7543859649122807



usaremos então n estimator igaula a 6

In [14]:
rf2 = RandomForestClassifier(warm_start=True,n_estimators=6,max_features='log2',criterion='entropy',
class_weight='balanced',bootstrap=True)

metricas_ran2 = cross_validate(rf2,x_nm, y_nm, cv=7, scoring=nome_metricas)
for met in metricas_ran:
  print(f'-{met}')
  print(f"-- {metricas_ran2[met]}")
  media = np.mean(metricas_ran2[met])
  desvio = np.std(metricas_ran2[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the r

-fit_time
-- [0.04904652 0.03809547 0.0734539  0.07102776 0.04423571 0.03455281
 0.04932594]
Média do fit_time: 0.051391158785138814
Desvio 0.014109919131744253
Intervalo [0.023,0.080]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.02057219 0.00744438 0.01509023 0.0165503  0.01834178 0.01227927
 0.0153861 ]
Média do score_time: 0.015094893319266183
Desvio 0.00394975756042371
Intervalo [0.007,0.023]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.92857143 0.82142857 0.82142857 0.81481481 0.7037037  0.66666667
 0.55555556]
Média do test_accuracy: 0.7588813303099018
Desvio 0.1150385551693525
Intervalo [0.529,0.989]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision_macro
-- [0.92857143 0.86842105 0.8368984  0.83235294 0.72222222 0.79545455
 0.63586957]
Média do test_precision_macro: 0.8028271644279374
Desvio 0.0898992694883838
Intervalo [0.623,0.983]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/home/gustavo/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:765: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the r

Comparando as métricas, temos que
- Random forest *sem otimização*:
    - acuracia: 0,758
    - precisão: 0,776
    - recall: 0,761

- Random forest *com otimização de grid search*:
    - acuracia: 0.790
    - precisão: 0.811
    - recall: 0.791

- Random forest *com otimização de random search*:
    - acuracia: 0,774
    - precisão: 0,821
    - recall: 0,776

podemos ver que com o grid search o random search  teve uma melhora, principalmente na precisão, embora com o grid search foi maior, mas é importante observar que o random search o processamento é mais rápido que o grid search.